In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import datetime
import dask.array

import sys
sys.path.append('../')
import snowFun

# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_dhdt = os.path.join(folder_AGVA, "dh_dt")

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi, drop='geometry')

In [2]:
# get list of all the glaciers we have data for
folder_source = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Annual AAs', 'csv')
folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')

# load rgi names that have been saved to the classified folder
rgis_to_analyze = list(set( [ i[3:17] for i in os.listdir(folder_source) if i!='merged.vrt' ] ))
rgis_to_analyze.sort()

print(len(rgis_to_analyze))

3023


In [3]:
# open dhdt
path_dhdt = os.path.join(folder_dhdt, "dh_01_rgi60_pergla_rates.csv")
dhdt_df = pd.read_csv(path_dhdt, usecols=["rgiid","period","dhdt"])

# dropping unwanted rows
dhdt_df = dhdt_df[dhdt_df['rgiid'].isin(rgis_to_analyze)]
dhdt_df = dhdt_df.rename(columns={'rgiid':'RGIId'})

# get only the rows and columns we are interested in (and merge)
dhdt_1520_df = dhdt_df[ dhdt_df['period'].isin(['2015-01-01_2020-01-01']) ].copy().rename(columns={'dhdt':'dhdt_1520'}).drop('period', axis=1)
dhdt_1020_df = dhdt_df[ dhdt_df['period'].isin(['2010-01-01_2020-01-01']) ].copy().rename(columns={'dhdt':'dhdt_1020'}).drop('period', axis=1)
dhdt_df = dhdt_1520_df.merge(dhdt_1020_df, on='RGIId')

print(dhdt_df.shape)
print(dhdt_df.head())

(3023, 3)
            RGIId  dhdt_1520  dhdt_1020
0  RGI60-01.00004    -1.0619    -0.8687
1  RGI60-01.00005     0.3284     0.2477
2  RGI60-01.00006     0.0705     0.2055
3  RGI60-01.00010    -1.1346    -1.0526
4  RGI60-01.00012    -1.0326    -0.9369


### Create a single dataframe that hold ELA and AAR info for each year for each glacier

In [4]:
# lastly, go through all the saved csvs, make a master csv with rgiid, median ela
all_files = [ i for i in os.listdir(os.path.join(folder_source)) if i[-3:]=='csv' ]

all_rgis = []
all_elas_2018 = []
all_elas_2019 = []
all_elas_2020 = []
all_elas_2021 = []
all_elas_2022 = []

all_aars_2018 = []
all_aars_2019 = []
all_aars_2020 = []
all_aars_2021 = []
all_aars_2022 = []

all_og_2018 = []
all_og_2019 = []
all_og_2020 = []
all_og_2021 = []
all_og_2022 = []

all_dates_2018 = []
all_dates_2019 = []
all_dates_2020 = []
all_dates_2021 = []
all_dates_2022 = []

i=0
for f in all_files:
    i+=1
#     if i>300: continue
    
    # open
    df = pd.read_csv(os.path.join(folder_source, f))
    
    # get rgi
    all_rgis.append( f[3:17] )
    
    # add data to lists
    all_elas_2018.append(int(df.loc[0]['ela']))
    all_elas_2019.append(int(df.loc[1]['ela']))
    all_elas_2020.append(int(df.loc[2]['ela']))
    all_elas_2021.append(int(df.loc[3]['ela']))
    all_elas_2022.append(int(df.loc[4]['ela']))
    
    all_aars_2018.append(round(df.loc[0]['aar'],4))
    all_aars_2019.append(round(df.loc[1]['aar'],4))
    all_aars_2020.append(round(df.loc[2]['aar'],4))
    all_aars_2021.append(round(df.loc[3]['aar'],4))
    all_aars_2022.append(round(df.loc[4]['aar'],4))
    
    all_og_2018.append(df.loc[0]['off_glacier'])
    all_og_2019.append(df.loc[1]['off_glacier'])
    all_og_2020.append(df.loc[2]['off_glacier'])
    all_og_2021.append(df.loc[3]['off_glacier'])
    all_og_2022.append(df.loc[4]['off_glacier'])
    
    all_dates_2018.append(df.loc[0]['date'])
    all_dates_2019.append(df.loc[1]['date'])
    all_dates_2020.append(df.loc[2]['date'])
    all_dates_2021.append(df.loc[3]['date'])
    all_dates_2022.append(df.loc[4]['date'])
    
    # get rid of the 'bad' years
#     df = df[df['ela']>0]
    
    # get median ela, aar
#     all_elas.append( np.nanmedian(df['ela']) )
#     all_aars.append( np.nanmedian(df['aar']) )

# save everything
# obs_df = pd.DataFrame({'RGIId':all_rgis, 'ela':all_elas, 'aar':all_aars}) 
obs_df = pd.DataFrame({'RGIId':all_rgis,
                       'ela_2018':all_elas_2018, 'ela_2019':all_elas_2019, 'ela_2020':all_elas_2020, 'ela_2021':all_elas_2021, 'ela_2022':all_elas_2022,
                       'aar_2018':all_aars_2018, 'aar_2019':all_aars_2019, 'aar_2020':all_aars_2020, 'aar_2021':all_aars_2021, 'aar_2022':all_aars_2022,
                       'off_glacier_2018':all_og_2018, 'off_glacier_2019':all_og_2019, 'off_glacier_2020':all_og_2020, 'off_glacier_2021':all_og_2021, 'off_glacier_2022':all_og_2022,
                       'date_2018':all_dates_2018, 'date_2019':all_dates_2019, 'date_2020':all_dates_2020, 'date_2021':all_dates_2021, 'date_2022':all_dates_2022,
                      }) 
print(obs_df.shape)
obs_df.head()

(3023, 21)


,RGIId,ela_2018,ela_2019,ela_2020,ela_2021,ela_2022,aar_2018,aar_2019,aar_2020,aar_2021,...,off_glacier_2018,off_glacier_2019,off_glacier_2020,off_glacier_2021,off_glacier_2022,date_2018,date_2019,date_2020,date_2021,date_2022
0,RGI60-01.00004,1840,2300,1860,1880,1900,0.5782,0.0950,0.4650,0.3593,...,False,True,False,False,False,2018-09-14,2019-08-08,2020-08-19,2021-08-02,2022-07-05
1,RGI60-01.00005,1790,2290,1880,2030,2040,0.6609,0.1291,0.4010,0.3408,...,False,True,False,False,False,2018-07-31,2019-08-08,2020-08-17,2021-07-16,2022-07-05
2,RGI60-01.00006,-9999,-9999,-9999,-9999,-9999,-9999.0000,-9999.0000,-9999.0000,-9999.0000,...,-9999,-9999,-9999,-9999,-9999,2018-01-01,2019-01-01,2020-01-01,2021-01-01,2022-01-01
3,RGI60-01.00010,1990,1990,1990,1990,1990,0.2207,0.0802,0.1974,0.1158,...,True,True,True,True,True,2018-11-09,2019-11-03,2020-10-28,2021-07-30,2022-10-31
4,RGI60-01.00012,1860,2010,1850,2010,1890,0.2657,0.0775,0.2493,0.0873,...,False,True,False,True,False,2018-09-09,2019-07-29,2020-08-14,2021-07-31,2022-07-11


In [5]:
# add rgi info (area, zmin, zmax, etc...) to df
full_data_df = rgi_gdf[rgi_gdf['RGIId'].isin(rgis_to_analyze)]
full_data_df = full_data_df.merge(obs_df, on='RGIId')
print(full_data_df.shape)
full_data_df.head()

(3023, 45)


,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,off_glacier_2018,off_glacier_2019,off_glacier_2020,off_glacier_2021,off_glacier_2022,date_2018,date_2019,date_2020,date_2021,date_2022
0,RGI60-01.00004,G213880E63381N,20090703,-9999999,-146.120,63.381,1,2,3.681,1273,...,False,True,False,False,False,2018-09-14,2019-08-08,2020-08-19,2021-08-02,2022-07-05
1,RGI60-01.00005,G212943E63551N,20090703,-9999999,-147.057,63.551,1,2,2.573,1494,...,False,True,False,False,False,2018-07-31,2019-08-08,2020-08-17,2021-07-16,2022-07-05
2,RGI60-01.00006,G213756E63571N,20090703,-9999999,-146.244,63.571,1,2,10.470,1201,...,-9999,-9999,-9999,-9999,-9999,2018-01-01,2019-01-01,2020-01-01,2021-01-01,2022-01-01
3,RGI60-01.00010,G212830E63513N,20090703,-9999999,-147.170,63.513,1,2,3.806,1280,...,True,True,True,True,True,2018-11-09,2019-11-03,2020-10-28,2021-07-30,2022-10-31
4,RGI60-01.00012,G213349E63366N,20090703,-9999999,-146.651,63.366,1,2,2.802,1318,...,False,True,False,True,False,2018-09-09,2019-07-29,2020-08-14,2021-07-31,2022-07-11


In [6]:
# open the derived ela aar for the 5-year average products
path_open = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Average AAs', 'average_aa_glacier_stats.csv')
average_df = pd.read_csv(path_open)
average_df = average_df.rename({'ela':'ela_avg_product',
                                'aar':'aar_avg_product',
                                'off_glacier':'off_glacier_avg_product',
                               }, axis=1)
average_df['percent_observed_average_product'] = (average_df['total_observed']/average_df['total_area']).round(4)
to_merge_df = average_df[['RGIId','ela_avg_product','aar_avg_product','off_glacier_avg_product','percent_observed_average_product']]
full_data_df = full_data_df.merge(to_merge_df, on='RGIId', how = 'outer')
print(full_data_df.shape)
full_data_df.head()

(3023, 49)


,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,off_glacier_2022,date_2018,date_2019,date_2020,date_2021,date_2022,ela_avg_product,aar_avg_product,off_glacier_avg_product,percent_observed_average_product
0,RGI60-01.00004,G213880E63381N,20090703,-9999999,-146.120,63.381,1,2,3.681,1273,...,False,2018-09-14,2019-08-08,2020-08-19,2021-08-02,2022-07-05,1870.0,0.3854,0.0,0.9841
1,RGI60-01.00005,G212943E63551N,20090703,-9999999,-147.057,63.551,1,2,2.573,1494,...,False,2018-07-31,2019-08-08,2020-08-17,2021-07-16,2022-07-05,2020.0,0.2739,0.0,0.9940
2,RGI60-01.00006,G213756E63571N,20090703,-9999999,-146.244,63.571,1,2,10.470,1201,...,-9999,2018-01-01,2019-01-01,2020-01-01,2021-01-01,2022-01-01,NaN,NaN,NaN,NaN
3,RGI60-01.00010,G212830E63513N,20090703,-9999999,-147.170,63.513,1,2,3.806,1280,...,True,2018-11-09,2019-11-03,2020-10-28,2021-07-30,2022-10-31,1990.0,0.1103,1.0,0.9979
4,RGI60-01.00012,G213349E63366N,20090703,-9999999,-146.651,63.366,1,2,2.802,1318,...,False,2018-09-09,2019-07-29,2020-08-14,2021-07-31,2022-07-11,1910.0,0.1557,0.0,0.9766


In [7]:
# add dhdt info to df
full_dhdt_df = dhdt_df[dhdt_df['RGIId'].isin(rgis_to_analyze)]
full_data_df = full_data_df.merge(full_dhdt_df, on='RGIId')
print(full_data_df.shape)
full_data_df.head()

(3023, 51)


,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,date_2019,date_2020,date_2021,date_2022,ela_avg_product,aar_avg_product,off_glacier_avg_product,percent_observed_average_product,dhdt_1520,dhdt_1020
0,RGI60-01.00004,G213880E63381N,20090703,-9999999,-146.120,63.381,1,2,3.681,1273,...,2019-08-08,2020-08-19,2021-08-02,2022-07-05,1870.0,0.3854,0.0,0.9841,-1.0619,-0.8687
1,RGI60-01.00005,G212943E63551N,20090703,-9999999,-147.057,63.551,1,2,2.573,1494,...,2019-08-08,2020-08-17,2021-07-16,2022-07-05,2020.0,0.2739,0.0,0.9940,0.3284,0.2477
2,RGI60-01.00006,G213756E63571N,20090703,-9999999,-146.244,63.571,1,2,10.470,1201,...,2019-01-01,2020-01-01,2021-01-01,2022-01-01,NaN,NaN,NaN,NaN,0.0705,0.2055
3,RGI60-01.00010,G212830E63513N,20090703,-9999999,-147.170,63.513,1,2,3.806,1280,...,2019-11-03,2020-10-28,2021-07-30,2022-10-31,1990.0,0.1103,1.0,0.9979,-1.1346,-1.0526
4,RGI60-01.00012,G213349E63366N,20090703,-9999999,-146.651,63.366,1,2,2.802,1318,...,2019-07-29,2020-08-14,2021-07-31,2022-07-11,1910.0,0.1557,0.0,0.9766,-1.0326,-0.9369


In [8]:
# add distance from ocean info to df
ocean_df = pd.read_csv(os.path.join(folder_AGVA, "Derived products", "S2", "distance_from_ocean.csv"))
full_data_df = full_data_df.merge(ocean_df, on='RGIId')
print(full_data_df.shape)
full_data_df.head()

(3023, 52)


,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,date_2020,date_2021,date_2022,ela_avg_product,aar_avg_product,off_glacier_avg_product,percent_observed_average_product,dhdt_1520,dhdt_1020,distance_from_ocean
0,RGI60-01.00004,G213880E63381N,20090703,-9999999,-146.120,63.381,1,2,3.681,1273,...,2020-08-19,2021-08-02,2022-07-05,1870.0,0.3854,0.0,0.9841,-1.0619,-0.8687,249.19
1,RGI60-01.00005,G212943E63551N,20090703,-9999999,-147.057,63.551,1,2,2.573,1494,...,2020-08-17,2021-07-16,2022-07-05,2020.0,0.2739,0.0,0.9940,0.3284,0.2477,255.38
2,RGI60-01.00006,G213756E63571N,20090703,-9999999,-146.244,63.571,1,2,10.470,1201,...,2020-01-01,2021-01-01,2022-01-01,NaN,NaN,NaN,NaN,0.0705,0.2055,267.31
3,RGI60-01.00010,G212830E63513N,20090703,-9999999,-147.170,63.513,1,2,3.806,1280,...,2020-10-28,2021-07-30,2022-10-31,1990.0,0.1103,1.0,0.9979,-1.1346,-1.0526,249.05
4,RGI60-01.00012,G213349E63366N,20090703,-9999999,-146.651,63.366,1,2,2.802,1318,...,2020-08-14,2021-07-31,2022-07-11,1910.0,0.1557,0.0,0.9766,-1.0326,-0.9369,239.80


In [9]:
# drop geometry column prior to saving
good_cols = [i for i in full_data_df.columns if i != "geometry"]
full_data_df = full_data_df[good_cols]
full_data_df.head()

,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,date_2020,date_2021,date_2022,ela_avg_product,aar_avg_product,off_glacier_avg_product,percent_observed_average_product,dhdt_1520,dhdt_1020,distance_from_ocean
0,RGI60-01.00004,G213880E63381N,20090703,-9999999,-146.120,63.381,1,2,3.681,1273,...,2020-08-19,2021-08-02,2022-07-05,1870.0,0.3854,0.0,0.9841,-1.0619,-0.8687,249.19
1,RGI60-01.00005,G212943E63551N,20090703,-9999999,-147.057,63.551,1,2,2.573,1494,...,2020-08-17,2021-07-16,2022-07-05,2020.0,0.2739,0.0,0.9940,0.3284,0.2477,255.38
2,RGI60-01.00006,G213756E63571N,20090703,-9999999,-146.244,63.571,1,2,10.470,1201,...,2020-01-01,2021-01-01,2022-01-01,NaN,NaN,NaN,NaN,0.0705,0.2055,267.31
3,RGI60-01.00010,G212830E63513N,20090703,-9999999,-147.170,63.513,1,2,3.806,1280,...,2020-10-28,2021-07-30,2022-10-31,1990.0,0.1103,1.0,0.9979,-1.1346,-1.0526,249.05
4,RGI60-01.00012,G213349E63366N,20090703,-9999999,-146.651,63.366,1,2,2.802,1318,...,2020-08-14,2021-07-31,2022-07-11,1910.0,0.1557,0.0,0.9766,-1.0326,-0.9369,239.80


In [10]:
out_path = os.path.join(folder_save, 'Final_Data_Table.csv')
full_data_df.to_csv(out_path, index=False)